The purpose of this notebook is to take the combined data frame from the controller and process the variables within it into values that can be useful in the model and easily understood for analysis purposes.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob

In [5]:
df = pd.read_csv('contdata_processed', parse_dates=[['Date','Time']])

In [20]:
dfa=pd.read_csv('contdata_parsed.csv')
dfa.columns = ['number', 'Date_Time','unnamed','ducttempC','fankWh','fan%act','fan%ct','fandp','filterdp']
dfb = dfa.drop(['number'], axis=1)
dfc=dfb.drop(['unnamed'], axis=1)
#df.to_csv('contdata_parsed.csv')
#dfb = dfa.drop(['Unnamed:0'], axis=1)
dfc.to_csv('dfc.csv')
dfc.head()

,Date_Time,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp
0,3/25/2019 12:51,3.9,1302.0,75.0,77.0,89.0,41.0
1,3/25/2019 20:14,4.4,3174.0,90.0,90.0,117.0,54.0
2,3/25/2019 20:15,4.3,3178.0,90.0,89.0,117.0,55.0
3,3/25/2019 20:16,4.2,3182.0,85.0,87.0,112.0,53.0
4,3/25/2019 20:17,4.2,3186.0,90.0,86.0,111.0,52.0


In [2]:
dfx=pd.read_csv('dfc.csv')
dfx.head()

,Date_Time,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp
0,3/25/2019 13:00,4.0,1331.0,78.0,78.0,93.0,43.0
1,3/25/2019 13:01,4.0,1334.0,77.0,78.0,93.0,43.0
2,3/25/2019 13:02,4.0,1337.0,77.0,79.0,94.0,43.0
3,3/25/2019 13:03,4.1,1341.0,80.0,79.0,95.0,44.0
4,3/25/2019 13:04,4.1,1344.0,78.0,80.0,94.0,44.0


In [3]:
#covert SI to IP units, calc fan airflow the 5835 m^3/hr is the known maximum airflow from the measuring device calibration
dfx['ducttempF']= dfx['ducttempC'] * (9/5) +32
dfx['fanm3hr']= dfx['fan%act']/100*5835
dfx['fancfm']=dfx['fanm3hr']/1.69901082

dfx.head()

,Date_Time,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp,ducttempF,fanm3hr,fancfm
0,3/25/2019 13:00,4.0,1331.0,78.0,78.0,93.0,43.0,39.20,4551.30,2678.794006
1,3/25/2019 13:01,4.0,1334.0,77.0,78.0,93.0,43.0,39.20,4492.95,2644.450493
2,3/25/2019 13:02,4.0,1337.0,77.0,79.0,94.0,43.0,39.20,4492.95,2644.450493
3,3/25/2019 13:03,4.1,1341.0,80.0,79.0,95.0,44.0,39.38,4668.00,2747.481031
4,3/25/2019 13:04,4.1,1344.0,78.0,80.0,94.0,44.0,39.38,4551.30,2678.794006


In [4]:
#Calculate the kWh difference for each time point
dfx['kWhdiff'] = dfx['fankWh'].diff()
dfx.head()
#describe the data to find issues with sensor readings to prepare to mask and eventually replace bad values
dfx.describe()

,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp,ducttempF,fanm3hr,fancfm,kWhdiff
count,296339.000000,296339.000000,296339.000000,296339.000000,296339.000000,296339.000000,296339.000000,296339.00000,296339.000000,296331.000000
mean,18.342011,2900.297764,68.895282,96.700350,106.399532,66.558188,65.015620,4020.03971,2366.106009,0.015135
std,7.444864,1753.725674,14.361682,14.338663,17.457635,18.722811,13.400754,838.00413,493.230602,138.178106
min,-99.900000,0.000000,0.000000,20.000000,0.000000,0.000000,-147.820000,0.00000,0.000000,-6329.000000
25%,14.200000,1370.000000,60.000000,100.000000,106.000000,57.000000,57.560000,3501.00000,2060.610774,4.000000
50%,19.200000,2869.000000,68.000000,100.000000,110.000000,66.000000,66.560000,3967.80000,2335.358877,4.000000
75%,23.500000,4398.000000,82.000000,100.000000,114.000000,81.000000,74.300000,4784.70000,2816.168057,4.000000
max,35.200000,6330.000000,128.000000,100.000000,352.000000,104.000000,95.360000,7468.80000,4395.969650,3883.000000


In [5]:
#replace numbers over 10 and less than 0 with nan, from the counter resetting early attempts at this
#dfx['kWhdiff'].replace(-6329, np.nan)
#dfx.describe()
#dfx.head()
dfxz = dfx.copy()

In [6]:
#dfxz['kWhdiff'].iloc[:, (df>=10) & (df<=0)] = np.nan


In [9]:
# Final Masking conditions for the kWh difference and temperature readings for both SI and IP units
dfxz['kWhdiff'].mask(dfxz['kWhdiff'].between(-10000, 0), inplace=True)
dfxz['kWhdiff'].mask(dfxz['kWhdiff'].between(10, 5000), inplace=True)
dfxz['ducttempC'].mask(dfxz['ducttempC'].between(-10000, -29), inplace=True)
dfxz['ducttempF'].mask(dfxz['ducttempF'].between(-10000, -20), inplace=True)

In [10]:
dfxz.describe()

,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp,ducttempF,fanm3hr,fancfm,kWhdiff
count,296333.000000,296339.000000,296339.000000,296339.000000,296339.000000,296339.000000,296333.000000,296339.00000,296339.000000,232354.000000
mean,18.344405,2900.297764,68.895282,96.700350,106.399532,66.558188,65.019930,4020.03971,2366.106009,4.092471
std,7.425902,1753.725674,14.361682,14.338663,17.457635,18.722811,13.366624,838.00413,493.230602,0.635219
min,-5.200000,0.000000,0.000000,20.000000,0.000000,0.000000,22.640000,0.00000,0.000000,1.000000
25%,14.200000,1370.000000,60.000000,100.000000,106.000000,57.000000,57.560000,3501.00000,2060.610774,4.000000
50%,19.200000,2869.000000,68.000000,100.000000,110.000000,66.000000,66.560000,3967.80000,2335.358877,4.000000
75%,23.500000,4398.000000,82.000000,100.000000,114.000000,81.000000,74.300000,4784.70000,2816.168057,4.000000
max,35.200000,6330.000000,128.000000,100.000000,352.000000,104.000000,95.360000,7468.80000,4395.969650,8.000000


In [11]:
#fill the missing values, the last line is to fillin the very first value in the data frame
dfxz1=dfxz.copy()
dfxz1['kWhdiff'] = dfxz['kWhdiff'].fillna(dfxz['kWhdiff'].interpolate(method='linear'))
dfxz1['ducttempC'] = dfxz['ducttempC'].fillna(dfxz['ducttempC'].interpolate(method='linear'))
dfxz1['ducttempF'] = dfxz['ducttempF'].fillna(dfxz['ducttempF'].interpolate(method='linear'))
dfxz1=dfxz1.fillna(dfxz.interpolate(method='linear'))

In [12]:
dfxz1.describe()


,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp,ducttempF,fanm3hr,fancfm,kWhdiff
count,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316449.000000
mean,18.678553,3103.110398,68.076650,96.910049,107.190482,68.476769,65.621396,3972.272547,2337.991319,4.123405
std,7.300629,1868.352019,14.256654,13.898887,17.173486,19.576129,13.141132,831.875757,489.623578,0.705986
min,-5.200000,0.000000,0.000000,20.000000,0.000000,0.000000,22.640000,0.000000,0.000000,1.000000
25%,14.600000,1472.000000,58.000000,100.000000,106.000000,57.000000,58.280000,3384.300000,1991.923748,4.000000
50%,19.800000,3074.000000,67.000000,100.000000,111.000000,67.000000,67.640000,3909.450000,2301.015364,4.000000
75%,23.600000,4717.000000,81.000000,100.000000,115.000000,83.000000,74.480000,4726.350000,2781.824544,4.500000
max,35.200000,6330.000000,128.000000,100.000000,352.000000,104.000000,95.360000,7468.800000,4395.969650,8.000000


In [13]:
#realized the fan kWh meter readings is one pulse for every W*h not kW*h
#this was verified as the fan is supplied 240VAC and draws an estimated 1.2A based on the spec sheet
dfxz1['fanwatts']=dfxz1['kWhdiff']/0.016667

In [14]:
#convert the Pascal static pressure readings to inches of water column
dfxz1['faninwc']=dfxz1['fandp']/248.84
dfxz1['filterinwc']=dfxz1['filterdp']/248.84

In [15]:
dfxz1.describe()

,ducttempC,fankWh,fan%act,fan%ct,fandp,filterdp,ducttempF,fanm3hr,fancfm,kWhdiff,fanwatts,faninwc,filterinwc
count,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316450.000000,316449.000000,316449.000000,316450.000000,316450.000000
mean,18.678553,3103.110398,68.076650,96.910049,107.190482,68.476769,65.621396,3972.272547,2337.991319,4.123405,247.399373,0.430761,0.275184
std,7.300629,1868.352019,14.256654,13.898887,17.173486,19.576129,13.141132,831.875757,489.623578,0.705986,42.358286,0.069014,0.078670
min,-5.200000,0.000000,0.000000,20.000000,0.000000,0.000000,22.640000,0.000000,0.000000,1.000000,59.998800,0.000000,0.000000
25%,14.600000,1472.000000,58.000000,100.000000,106.000000,57.000000,58.280000,3384.300000,1991.923748,4.000000,239.995200,0.425977,0.229063
50%,19.800000,3074.000000,67.000000,100.000000,111.000000,67.000000,67.640000,3909.450000,2301.015364,4.000000,239.995200,0.446070,0.269249
75%,23.600000,4717.000000,81.000000,100.000000,115.000000,83.000000,74.480000,4726.350000,2781.824544,4.500000,269.994600,0.462144,0.333548
max,35.200000,6330.000000,128.000000,100.000000,352.000000,104.000000,95.360000,7468.800000,4395.969650,8.000000,479.990400,1.414564,0.417939


In [16]:
#output the final data to be inputed into the combining and modeling code
dfxz1.to_csv('contdata_finalproc.csv')